In [1]:
import cx_Oracle
import os
import pandas as pd
import xlwings as xw
import json
import datetime as dt
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']
LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
cx_Oracle.init_oracle_client(lib_dir=LOCATION)

connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()


In [2]:
# 해외품목 STATE에 관하여...
# Data Insert => 한국으로 들어올예정 > Scheduled SCHEDULED
# InboundReday => 실물 입고 예정(검수전) - > HOLDING
# INSPECTION DONE => 검수완료 -> GOOD
PRODUCT_STATE = ['SCHEDULED','HOLDING']

#### INSPECTION DONE
1. 상품상태 이상 없음 
    - GOOD
        - GOOD_WAREHOUSE 
            - 창고입고완료 이상 X
        - GOOD_ELSE
            - 외부창고 입고완료 이상 X
2. 상품상태 이상 있음
    1. 파손(DAMAGED)
        - DMG_WET
            - 젖음
        - DMG_TEMPARTURE
            - 온도 유지실패
        - DMG_ONLYOUTER
            - 겉포장만 손상
        - DMG_IMPACT
            - 충격으로인한 손상
        - DMG_ELSE
            - 기타손상
    2. 상품이상(WRONG)
        - WR_SHORT 
            - 수량부족
        - WR_OVER
            - 수량과다
        - WR_PRODUCT
            - 상이한제품
        - WR_ELSE
            - 기타상품이상
3. 반납상품
    - RETURN
    
4. 배송
    SHIP_CONFIRM
        - 배송팀에 인계완료 약속시간까지 배송예정
    POD_DONE
        - POD 완료 모든 과정 완료

In [162]:
# edit_mode에서만 사용 가능
sel_sht = wb_cy.selection.sheet
status_cel = sel_sht.range("H4").value
selected_cel_one = wb_cy.selection.address
selected_cel_visible = wb_cy.selection.api.SpecialCells(12).Address
selected_cel_value = wb_cy.selection.value

if selected_cel_one.count(':') == 0:
    selected_cel = selected_cel_one
else :
    selected_cel = selected_cel_visible

if status_cel != 'edit_mode':
    wb_cy.app.alert("edimt_mode에서만 진행 가능합니다. Mode STATUS를 확인해주세요.","WARNING")
#     return
sel_sht.range("U3").select()
wb_cy.app.alert("검수완료할 품목의 STATE 컬럼을 선택하여 진행해주세요.","INFO")
dollar_cnt = selected_cel.count("$")
R_count = selected_cel.count("R")
is_R_selected = dollar_cnt/2 == R_count
if is_R_selected == False :
    wb_cy.app.alert("STATE컬럼 이외의 다른 셀이 선택되어있습니다. 다시 선택해주세요.","WARNING")
#     return


In [165]:
si_idx_address = selected_cel.replace("R",'A').split(',')
tmp_list = []
si_idx_list=[]
if type(si_idx_address ) is list :
    for adds in si_idx_address:
        tmp_val = sel_sht.range(adds).options(ndim=1,numbers=int).value
        tmp_list.append(tmp_val)
for i in range(len(tmp_list)):
    if tmp_list[i] == None:
        wb_cy.app.alert("SI_INDEX값이 없는 품목이 선택되어있습니다. 선택행을 확인해주세요","Quit")
#         return
    
    si_idx_list = si_idx_list + tmp_list[i]

for idx in si_idx_list:
    each_state = cursor.execute(f"select state from shipment_information where si_index = {idx}").fetchone()[0]
    if each_state != 'HOLDING':
        wb_cy.app.alert(f"SI_INDEX : {idx} STATE : {each_state}, STATE는 HOLDING 상태에서만 진행 가능합니다. 매서드를 종료합니다.","Quit")
#         return
    
selected_cell_in_xl = sel_sht.range("V3")
to_status = sel_sht.range("V4")
selected_cell_in_xl.clear_contents()
to_status.clear()
selected_cell_in_xl .value = selected_cel

state_list = list(pd.DataFrame(cursor.execute('select * from INSPECTION_CODE').fetchall())[0])[2:]
state_list = ','.join(state_list)

to_status.font.size = 16
to_status.api.Borders.LineStyle = 1 
to_status.api.Validation.Add(Type=3, Formula1=state_list)






In [172]:
def select_cells_for_inspection():
    # edit_mode에서만 사용 가능
    sel_sht = wb_cy.selection.sheet
    status_cel = sel_sht.range("H4").value
    selected_cel_one = wb_cy.selection.address
    selected_cel_visible = wb_cy.selection.api.SpecialCells(12).Address
    selected_cel_value = wb_cy.selection.value

    if selected_cel_one.count(':') == 0:
        selected_cel = selected_cel_one
    else :
        selected_cel = selected_cel_visible

    if status_cel != 'edit_mode':
        wb_cy.app.alert("edimt_mode에서만 진행 가능합니다. Mode STATUS를 확인해주세요.","WARNING")
        return
    sel_sht.range("U3").select()
    wb_cy.app.alert("검수완료할 품목의 STATE 컬럼을 선택하여 진행해주세요.","INFO")
    dollar_cnt = selected_cel.count("$")
    R_count = selected_cel.count("R")
    is_R_selected = dollar_cnt/2 == R_count
    if is_R_selected == False :
        wb_cy.app.alert("STATE컬럼 이외의 다른 셀이 선택되어있습니다. 다시 선택해주세요.","WARNING")
        return
    si_idx_address = selected_cel.replace("R",'A').split(',')
    tmp_list = []
    si_idx_list=[]
    if type(si_idx_address ) is list :
        for adds in si_idx_address:
            tmp_val = sel_sht.range(adds).options(ndim=1,numbers=int).value
            tmp_list.append(tmp_val)
    for i in range(len(tmp_list)):
        if tmp_list[i] == None:
            wb_cy.app.alert("SI_INDEX값이 없는 품목이 선택되어있습니다. 선택행을 확인해주세요","Quit")
            return

        si_idx_list = si_idx_list + tmp_list[i]

    for idx in si_idx_list:
        each_state = cursor.execute(f"select state from shipment_information where si_index = {idx}").fetchone()[0]
        if each_state != 'HOLDING':
            wb_cy.app.alert(f"SI_INDEX : {idx} STATE : {each_state}, STATE는 HOLDING 상태에서만 진행 가능합니다. 매서드를 종료합니다.","Quit")
            return

    selected_cell_in_xl = sel_sht.range("V3")
    to_status = sel_sht.range("V4")
    selected_cell_in_xl.clear_contents()
    to_status.clear_contents()
    
    selected_cell_in_xl .value = selected_cel

    state_list = list(pd.DataFrame(cursor.execute('select * from INSPECTION_CODE').fetchall())[0])[2:]
    state_list = ','.join(state_list)

    to_status.font.size = 16
    to_status.api.Borders.LineStyle = 1 
    to_status.api.Validation.Add(Type=3, Formula1=state_list)


In [191]:
answer = wb_cy.app.alert("선택하신 상태로 검수를 완료하시겠습니까","CONFIRM",buttons="yes_no_cancel")
if answer != "yes":
    wb_cy.app.alert("종료합니다.","Quit")

'yes'

In [192]:
['sdf',None,'123'].count(None)

1

In [194]:
str(to_status)

'<Range [cytiva.xlsm]Shipment information!$V$4>'

In [ ]:
# inpection_done

In [186]:
def inpection_done():
    selected_cell_in_xl.value.replace("R",'A').split(',')
    tmp_list=[]
    for adds in si_idx_address:
        tmp_val = sel_sht.range(adds).options(ndim=1,numbers=int).value
        tmp_list.append(tmp_val)
    for i in range(len(tmp_list)):
        if tmp_list[i] == None:
            wb_cy.app.alert("SI_INDEX값이 없는 품목이 선택되어있습니다. 선택행을 확인해주세요","Quit")
            return

        si_idx_list = si_idx_list + tmp_list[i]

    update_status(si_idx_list,to_status.value)
    bring_data_from_db()

[11006,
 11010,
 11011,
 11012,
 11013,
 11014,
 11006,
 11010,
 11011,
 11012,
 11013,
 11014,
 11006,
 11010,
 11011,
 11012,
 11013,
 11014]

In [188]:
update_status(self,si_idx_list,to_status.value)

'GOOD_WR'

In [ ]:
def select_cell():
    sel_cells = wb_cy.selection
    sel_sht = wb_cy.selection.sheet
    # 선택한 셀의 row번호가 10미만이면 종료 ==> table값은 row가 10부터 시작이기 때문
    if wb_cy.selection.row < 10 :
        wb_cy.app.alert("선택한 셀은 바꿀 수 없습니다. 매서드를 종료합니다.","Change Cell WARNING")
        return None
    address_cell = sel_sht.range("E3")
    from_cell = sel_sht.range("E4")
    # 선택한셀의 value가 list type 이면 두 개이상의 셀을 선택 했다는 것 ==> 종료
    if type(sel_cells.value) is list :
        wb_cy.app.alert("하나의 셀만 선택 후 진행해주세요. 두 개 이상은 불가합니다.","Change Cell WARNING")
        return None
    
    address_cell.value = str(sel_cells.address)
    from_cell.value = sel_cells.value

def change_cell():
    sel_sht = wb_cy.selection.sheet
    address_cell = sel_sht.range("E3")
    change_cell_list = sel_sht.range("E3:E4")
    tb_name = sel_sht.range("D5").value
    idx_col_name = sel_sht.range("A9").value
    cur = DataWarehouse()
    
    # 셀주소가 빈값이면 중지한다.
    if address_cell.value == None:
        wb_cy.app.alert("바꿀 셀이 없습니다 매서드를 종료합니다","Change Cell WARNING")
        return None
    
    xl_from_cell = sel_sht.range(address_cell.value)
    to_cell = wb_cy.app.api.InputBox("바꿀 내용을 입력 해주세요", "Change Cell Input", Type=2)
    # to_cell == False면 입력을 취소 했다는 뜻이므로 바꿀 뜻이 없는 것으로 간주하고 주소와 바뀔 값들의 form을 지운다.
    if to_cell == False :
        wb_cy.app.alert("취소를 선택하셨습니다. 셀 변경을 취소합니다.","Change Cell WARNING")
        change_cell_list.clear_contents()
        return None